Welcome!

This recipe provides a guide to **fine-tuning granite-3.1-1b-a400m-instruct**


There are several distinct types of fine-tuning, each suited to different use cases. Instruction tuning aligns the model with general task-following behavior, conversation tuning optimizes it for dialogue and multi-turn interactions, and domain-specific tuning adapts the model to specialized fields.

This recipe explores domain specific training and fine-tunes the model to perform better on math reasoning tasks.


**THIS NOTEBOOK WORKS IN LINUX/WINDOWS ENVIRONMENT AND REQUIRES A CUDA-ENABLED GPU (NVIDIA GPU).**

Please refer to the Unsloth system requirements [here](https://docs.unsloth.ai/get-started/beginner-start-here/unsloth-requirements).

This notebook is designed to work with the free level of GPU available from Google Colab, the T4 GPU. You should not need to pay for GPU time to run this simple fine-tuning demo.

If you want to fine-tune using larger datasets, you will need a machine or runtime with a large GPU to perform tuning. Your local computer can't run this notebook without a CUDA-enabled GPU.

**Troubleshooting:**

- **Verify runtime type**: Under the "Runtime" menu, select "Change runtime type." The "Hardware accelerator" field must be set to T4 GPU.
- **Verify runtime is connected**: In the top right, you should see the connection status for the T4 runtime. You should see a green checkmark, next to "T4", and "RAM Disk.".
- **Verify T4 GPU is only connected to one Colab session**: If you see the word "Connecting" in the top right and it doesn't seem to be resolving, click the arrow dropdown next to it and choose "Manage sessions". If there is an active session already (say, from another run of the notebook in a different browser window, or from using another notebook), you will need to disconnect the session. Click "Terminate other sessions" to do so.

## Install Dependencies

In [ ]:
# @title Titre par défaut
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu121

!pip install datasets evaluate
!pip install "transformers>=4.44.0"
!pip install "peft>=0.12.0"
!pip install "trl>=0.9.6"
!pip install accelerate bitsandbytes
!pip install vllm


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 219.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 201.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 232.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 132.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Fine Tuning Granite Model


### Import packages

In [ ]:
import re
import os, torch, random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
from ragas.metrics import answer_correctness
from ragas import evaluate

INFO 09-13 12:51:25 [__init__.py:241] Automatically detected platform cuda.


In [ ]:
SEED = 42
random.seed(SEED); torch.manual_seed(SEED)


### Prepare the Math Dataset and reward functions

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore


# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    scores = []

    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nExtracted:\n{extracted_responses[0]}")

    for i, (r, a) in enumerate(zip(extracted_responses, answer)):
        data_samples = {
            'question': [q],
            'answer': [r],
            'ground_truth': [a]
        }
        dataset = Dataset.from_dict(data_samples)
        score = evaluate(dataset, metrics=[answer_correctness])
        scores.append(score["answer_correctness"][0])

    return scores



def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [ ]:


# si tu utilises get_gsm8k_questions
train_dataset = get_gsm8k_questions(split="train")
test_dataset = get_gsm8k_questions(split="test")

print("Taille du train_dataset :", len(train_dataset))
print("Taille du test_dataset  :", len(test_dataset))

small_test_dataset = test_dataset.select(range(100))
small_train_dataset = train_dataset.select(range(100))

Taille du train_dataset : 7473
Taille du test_dataset  : 1319


### Loading the base model using Lora config

In [ ]:
model_id = "granite-3.1-1b-a400m-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, load_in_4bit=True, dtype=torch.bfloat16, device_map="auto"
)
peft_cfg = LoraConfig(
    r=32, lora_alpha=16, lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",    # self-attn
        "router.layer"                             # router
    ],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_cfg)
print(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteMoeForCausalLM(
      (model): GraniteMoeModel(
        (embed_tokens): Embedding(49152, 1024, padding_idx=0)
        (layers): ModuleList(
          (0-23): 24 x GraniteMoeDecoderLayer(
            (self_attn): GraniteMoeAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
 

In [ ]:
tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tok.padding_side = "left"
tok.pad_token = tok.eos_token
tok.chat_template = """{% for message in messages %}
{% if message['role'] == 'system' %}
System: {{ message['content'] }}
{% elif message['role'] == 'user' %}
User: {{ message['content'] }}
{% elif message['role'] == 'assistant' %}
Assistant: {{ message['content'] }}
{% endif %}
{% endfor %}"""

### Evaluate model before finetuning using the metric answer_correctness available in RAGAS

In [ ]:
os.environ["OPENAI_API_KEY"] =""


In [ ]:
def evaluation(model, tokenizer, dataset):
    questions=[]
    answers=[]
    grounds_truths=[]
    total = len(dataset)
    for sample in dataset:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": sample['question']}
        ]

        # Tokenize and return both input_ids and attention_mask
        encoding = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            return_dict=True
        ).to("cuda")

        # Generate using both input_ids and attention_mask
        output_ids = model.generate(
            encoding["input_ids"],
            attention_mask=encoding["attention_mask"],
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=256,
            use_cache=True
        )

        # Decode only the newly generated tokens
        response = tokenizer.decode(
            output_ids[0][encoding["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        # Extraire la réponse du XML
        extracted_answer = extract_xml_answer(response)
        print("response", response)



        questions.append(sample['question'])
        answers.append(extracted_answer)
        grounds_truths.append(sample['answer'])


    data_samples = {
            'question': questions,
            'answer': answers,
            'ground_truth': grounds_truths
        }
    print(data_samples)
    dataset = Dataset.from_dict(data_samples)
    score = evaluate(dataset,metrics=[answer_correctness])
    score.to_pandas()
    print(score)
    return score


before_score = evaluation(model, tok, test_dataset)
print("Exactitude avant fine-tuning:", before_score)


response 
Answer:

Janet's ducks lay 16 eggs per day, so she produces 16 eggs per day. She eats three for breakfast every morning, so she eats 3 eggs per day. She bakes muffins for her friends, and she sells the remainder at the farmers' market.

The farmers' market sells eggs for $2 per fresh duck egg, so she makes $2 for every egg she sells.

She eats three eggs per day, so she eats 3 eggs per day.

She bakes muffins for her friends, and she sells the remainder at the farmers' market.

The farmers' market sells eggs for $2 per fresh duck egg, so she makes $2 for every egg she sells.

She eats three eggs per day, so she eats 3 eggs per day.

She bakes muffins for her friends, and she sells the remainder at the farmers' market.

The farmers' market sells eggs for $2 per fresh duck egg, so she makes $
response 
<reasoning>
The robe takes 2 bolts of blue fiber and half that much white fiber.  So it takes 2/2 = 1 bolt of blue fiber and 1/2 = 0.5 bolt of white fiber.  So it takes 1 + 0.5 =

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'answer_correctness': 0.4412}
Exactitude avant fine-tuning: {'answer_correctness': 0.4412}


### GRPO fine tuning

In [ ]:
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 2e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = 512,
    max_completion_length = 512,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 1000,
    save_steps = 1000,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
    generation_batch_size=2,
    wandb_log_unique_prompts=True,
)

trainer = GRPOTrainer(
    model = model,
    processing_class = tok,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [ ]:
trainer.train()

-------------------- Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? 
Answer:
72 
Extracted:
Could not find a solution because I have not yet asked some questions!
Can someone help me?
Altern
What is the cardinality of the set {1,, defined states?
Can I write end the optional input vector addition? 
			 he not read in free utility. Must order to exclude specificates if the topology that introduces negative acts, or large executed therefore cannot read the other anti- implications state in free object computer functions.
Phrased chance to expend denom earthiness in Windows' activity.
After executing quarter of discrete daily, she said he was well-educated agnostic hopefully. Native to Silicon I feel integrated won roads, union organisation, pandora to ohio,.
....
....
Denomities.'s effects are now encouraging talk pidgins, or cannot bill to apiag easelord happen. tendenc

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

-------------------- Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet? 
Answer:
5 
Extracted:
<start>
a)      Betty needs $28 more money for the wallet. </Children>

          prestashop webinar <Segoe UI would do better BEFor an apostrophe story?
During mdping BeTraNascance call-language WebinnovatelBusNot
listen Creativity <SheaMonkeyHarpLabourmontu<MabyrchaOpSerMonyAnyarrY
oweSeven week absence saleBooing <Shea MonkeyhAgourthydele"LessonEndedunityManchegoHonSunPairsSchedule"IslovY<Sevenes <Sevenes <whenajournifoods <es<unCoun2edendule <DueStonyM&policyId instancefound<MinedayosCloseEjYzsaCetan<outIsedis
ice (SheAglainanStory)Ice AguardVelcueoGroAv


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

-------------------- Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? 
Answer:
72 
Extracted:
Answer:
It is hard to give a solid answer given the empty square brackets. Seems like a reasonable guess would be that Natalia sold F0 ull) clips she sold sell altogether in April and May. Can altogether be hours and minutes or years.


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
### Save the model

In [ ]:
trainer.model.save_pretrained("outputs/granite31-moe-1b-grpo")
tok.save_pretrained("outputs/granite31-moe-1b-grpo")


### Evaluate the model after fine tuning

In [ ]:
# Chemin de ton dossier sauvegardé
model_path = "outputs/granite31-moe-1b-grpo"

# Rechargement
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)


before_score = evaluation(model, tokenizer, test_dataset)
print("Exactitude avant fine-tuning:", before_score)


### Conclusion

In this notebook, I demonstrated the different steps required to fine-tune the granite-3.1-1b-a400m-instruct model.

The process combined LoRA with GRPO, while applying reward functions that evaluated both the format and the quality of the responses using LLM-based metrics such as answer relevance.

I stopped the training early because it is resource- and time-consuming.
The main goal here was to illustrate the different steps involved.
Thanks!